#**DATA CONTEST SOLUTION**

> Here is the code for the data contest. I have explained everthing appropriately in comments.



Basic Analysis gave the following observations:
1. There are 10000 unique songs (platform_id's)
2. There are 14053 unique customer_id's.
3. There are features available for only 9981 songs in songs_data.
4. There are only 12799 customers in saved_for_later.

#1. **Preliminary steps**

In [ ]:
############## Importing Libraries ##################
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import sklearn 
import scipy
!pip install catboost
!pip install --upgrade catboost
!pip install lightgbm
from scipy import stats

Requirement already up-to-date: catboost in /usr/local/lib/python3.7/dist-packages (0.25.1)


In [ ]:
#Loading the training data
train_data = pd.read_csv("/content/train.csv")
train_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 710109 entries, 0 to 710108
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  710109 non-null  object
 1   song_id      710109 non-null  int64 
 2   score        710109 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 16.3+ MB


In [ ]:
#Loading the songs_data
songs_data = pd.read_csv("/content/songs.csv")
songs_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9981 entries, 0 to 9980
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   song_id             9981 non-null   int64  
 1   platform_id         9981 non-null   object 
 2   released_year       9965 non-null   float64
 3   language            8905 non-null   object 
 4   number_of_comments  9981 non-null   int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 390.0+ KB


In [ ]:
#Loading the song_labels
song_labels = pd.read_csv("/content/song_labels.csv")
song_labels.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999912 entries, 0 to 999911
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   platform_id  999912 non-null  object
 1   label_id     999912 non-null  int64 
 2   count        999912 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 22.9+ MB


In [ ]:
#loading_save_for_later
saveforlater = pd.read_csv("/content/save_for_later.csv")
saveforlater.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238880 entries, 0 to 238879
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  238880 non-null  object
 1   song_id      238880 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [ ]:
#Loading the test_data
test = pd.read_csv("/content/test.csv")
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 670681 entries, 0 to 670680
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   customer_id  670681 non-null  object
 1   song_id      670681 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 10.2+ MB


In [ ]:
#Saving the id's of test_row, to produce in outputs.
test_row_id = np.array([k for k in range(0, len(test))])

In [ ]:
#Initialising an array of np.nan's of test set's length, for later use.
score = np.array([np.nan]*len(test))

In [ ]:
#Adding the above arrays to the test data set. This is done to make the further steps easy, as explained later.
test["test_row_id"] = test_row_id
test["score"] = score

In [ ]:
#Reordering the rows of test set.
test = test[['test_row_id','customer_id', 'song_id', 'score']]
test

,test_row_id,customer_id,song_id,score
0,0,J24644,5304,NaN
1,1,G29701,123,NaN
2,2,M29977,11,NaN
3,3,H50752,1445,NaN
4,4,J42394,271,NaN
...,...,...,...,...
670676,670676,J20614,92,NaN
670677,670677,F11740,21,NaN
670678,670678,I43093,2294,NaN
670679,670679,J26924,1216,NaN


#2. **Preprocessing, Imputation of missing values**

In [ ]:
#This function sorts a pandas data frame as per the values in column with index i and returns sorted df
def sortdf(df, i):
  col = list(df.columns.values)
  l = df.values.tolist()
  sorted_l = sorted(l,key = lambda z : z[i])
  arr_s = np.array(sorted_l)
  res = pd.DataFrame(arr_s, columns = col)
  
  return res #the sorted data frame according to values in the given column index.

In [ ]:
#In song_labels, there are many labels assigned for the same song, by different users, frequency given by count.
#We will produce a unique label for a given song (two songs can have same label, but a song has only one label.)
#For this, we will take the weighted mean of label_id's using the counts and assign it as the label to the song.
#This may produce decimals, so we consider this feature as float value.
l = song_labels.values.tolist()
sorted_l = sorted(l,key=lambda z:z[0])
arr_sl = np.array(sorted_l)

nr_sl = np.shape(arr_sl)[0]

# Empty array of two columns
modified_sl = np.empty((0,2), (np.str_,int))
temp = 0

modified_sl = np.append(modified_sl,np.array([[arr_sl[0][0],0]]),axis=0)

modified_sl[0][1] = int(arr_sl[0][1]) * int(arr_sl[0][2])
cnt = 1

for i in range(1, nr_sl) :
  if (arr_sl[i][0] != arr_sl[i-1][0]) :
    sz = np.shape(modified_sl)[0]
    temp = int(modified_sl[sz-1][1])
    temp /= cnt
    modified_sl[sz-1][1] = temp
    modified_sl = np.append(modified_sl,np.array([[arr_sl[i][0],0]]),axis=0)
    modified_sl[sz][1] = int(arr_sl[i][1]) * int(arr_sl[i][2])
    cnt = 1
  else :
    sz = np.shape(modified_sl)[0]
    temp = int(modified_sl[sz - 1][1])
    temp += int(arr_sl[i][1]) * int(arr_sl[i][2])
    modified_sl[sz - 1][1] = temp
    cnt = cnt + 1
    if (i == nr_sl - 1) :
      temp = int(modified_sl[sz - 1][1])
      temp /= cnt
      modified_sl[sz - 1][1] = temp


In [ ]:
#We now convert the songs_data into array, and sort it according to the platform_id's.
#This is done because, we know that there is a bijection between platform_id and song_id, but we have only 9981 song_id's in songs_data file.
#But we have all 10000 unique platform_id's. So, we must first find an accurate match of the remaining 19 song_id's and platform_id's.
l1 = songs_data.values.tolist()
sorted_l1 = sorted(l1, key=lambda z:z[1])
arr_s = np.array(sorted_l1)

In [ ]:
#s_modified will give the features of 9981 songs with their platform_id's, using song_labels set.
#s_left over will give the label_id and platform_id of the other 19 songs.
s_modified = np.empty((0,6))
sl_leftover = np.empty((0,2))

sl_sz = np.shape(modified_sl)[0]  # 10000 only
s_sz = np.shape(arr_s)[0]         # 9981 only

j = 0 # for arr_s

for i in range(0, sl_sz) :
  if (j < s_sz and arr_s[j][1] == modified_sl[i][0]) :
    s_modified = np.append(s_modified, np.array([[arr_s[j][0], arr_s[j][1], arr_s[j][2], arr_s[j][3], arr_s[j][4], modified_sl[i][1]]]), axis=0)
    j = j + 1
  else :
    sl_leftover = np.append(sl_leftover, np.array([[modified_sl[i][0], modified_sl[i][1]]]), axis=0)

In [ ]:
#we will construct a data frame, songs19_df, to denote those 19 platform_id's and label_id's.
song_mod_df = pd.DataFrame(s_modified,columns=['song_id', 'platform_id','released_year','language','no.of_comments','label_id'])

songs19_df = pd.DataFrame(sl_leftover,columns = ['platform_id', 'label_id'])
song_mod_df.info()
songs19_df


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9981 entries, 0 to 9980
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   song_id         9981 non-null   object
 1   platform_id     9981 non-null   object
 2   released_year   9981 non-null   object
 3   language        9981 non-null   object
 4   no.of_comments  9981 non-null   object
 5   label_id        9981 non-null   object
dtypes: object(6)
memory usage: 468.0+ KB


,platform_id,label_id
0,Q34541,3358995.7
1,Q6485421,1599958.6
2,R16902,4339295.2
3,R90072,765840.34
4,S21323,1624348.5
5,S351023,1792782.6
6,S7402393,4932308.4
7,T11410104,308963.9
8,U1034605,280243.83
9,U1375,513049.99


In [ ]:
#The sortdf function has an issue with it.
#When the input dataframe has missing values (np.nan), it converts them into 'nan' string valued cstegorical variables.
#To remove that, we need to replace them by np.nan. We do that here.
for i in range(9981):
  if song_mod_df['released_year'][i] == 'nan':
     song_mod_df['released_year'][i] = np.nan
for i in range(9981):
  if song_mod_df['language'][i] == 'nan':
     song_mod_df['language'][i] = np.nan

In [ ]:
#This will add the platform_id correspondingly to the 9981 songs in song_labels dataset.
song_mod_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9981 entries, 0 to 9980
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   song_id         9981 non-null   object
 1   platform_id     9981 non-null   object
 2   released_year   9965 non-null   object
 3   language        8905 non-null   object
 4   no.of_comments  9981 non-null   object
 5   label_id        9981 non-null   object
dtypes: object(6)
memory usage: 468.0+ KB


In [ ]:
#We will create a dataframe with song_id an platform_id, wit hsong_id's sorted. It has 9981 rows.
#The middle values are missing, so the first and last values are 1 and 10000.
plat_song ={ "song_id" : song_mod_df["song_id"], "platform_id" : song_mod_df['platform_id']}
df = pd.DataFrame(plat_song)
for i in range(9981):
  df['song_id'][i] = int(df['song_id'][i])
df_sort = sortdf(df, 0)
df_sort

,song_id,platform_id
0,1,R2767052
1,2,S3
2,3,U41865
3,4,W2657
4,5,Q4671
...,...,...
9976,9996,V7130616
9977,9997,T208324
9978,9998,Q77431
9979,9999,S8565083


In [ ]:
#We will estimate the correlation between the 19 platform_id and song_id as follows:
#First, we will take a data set of 3 columns, which are the platform_id, label_id and song_id for 9981 songs in songs_data.
#Taking platform_id and label_id as features, we will fit a regression model to estimate the tagret - song_id.
#Now, using this model, we predict song_id's of the 19 other songs.
#We will sort these as per these predicted values and add the missing actial song_id's, so that the order is preserved.
#We use catboost for this becauce platform_id is a categorical feature. 
#Also, we use validation method to tune parameters of catboost.
from sklearn.model_selection import train_test_split
X = {'platform_id': song_mod_df['platform_id'],'label_id':song_mod_df['label_id'] }
X = pd.DataFrame(X)
y = pd.DataFrame(song_mod_df['song_id'])
X_test = songs19_df

from catboost import CatBoostRegressor

clf = CatBoostRegressor(iterations=5,cat_features= ['platform_id'],learning_rate=0.1)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

clf.fit(X_train, y_train, cat_features = ['platform_id'],eval_set=(X_val, y_val), verbose=False)

res = clf.predict(X_test)
X_test['song_id'] = res


s1 = set(train_data['song_id'])
s2 = set(songs_data['song_id'])
s3 = s1 - s2
s3 = list(sorted(s3))
X_test_sort = sortdf(X_test,2)
X_test_sort['song_id'] = s3
X_test_sort #This has the predicted mapping between the 19 song_id's and platform_id's, with label_id's.

,platform_id,label_id,song_id
0,U56405,7246933.2,819
1,S7402393,4932308.4,976
2,R16902,4339295.2,1724
3,Q34541,3358995.7,1754
4,X4352708,3517635.3,2142
5,Y7263429,3774969.9,2638
6,S351023,1792782.6,2951
7,S21323,1624348.5,3200
8,Q6485421,1599958.6,5612
9,Y2992069,1581898.9,5857


In [ ]:
concatenated_df = pd.concat([song_mod_df, X_test_sort])
concatenated_df.info()
#concatenated_df has 6 features, 10k rows for all unique songs, with missing values from song_labels data set.
concatenated_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 18
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   song_id         10000 non-null  object
 1   platform_id     10000 non-null  object
 2   released_year   9965 non-null   object
 3   language        8905 non-null   object
 4   no.of_comments  9981 non-null   object
 5   label_id        10000 non-null  object
dtypes: object(6)
memory usage: 546.9+ KB


,song_id,platform_id,released_year,language,no.of_comments,label_id
0,3753,P10,2005.0,eng,882,5.6
1,9019,P10090,1999.0,en-US,178,3677.8
2,7173,P102030,1935.0,eng,759,51773.3
3,43,P10210,1847.0,eng,31212,20781.8
4,5452,P1025250,2000.0,en-US,2441,275870.5
...,...,...,...,...,...,...
14,8055,T11410104,NaN,NaN,NaN,308963.9
15,8477,U1034605,NaN,NaN,NaN,280243.83
16,8665,U1375,NaN,NaN,NaN,513049.99
17,9534,V197436,NaN,NaN,NaN,463858.84


#3. **Filling missing rows in above dataset**

In [ ]:
#First we will check the percent of missing values.
#We can see that there are more than 10 percent.
#As we are not willing to lose the data of these songs, we will impute the missing values.
###### leaving out missing valued rows ###########
D = concatenated_df.dropna() 
D.head(7)

###### calculating percentage of data dropped #######
length = len(concatenated_df['song_id'])
length_new = len(D["song_id"])
per = (1 - length_new/length) *100
print("Percent of data dropped is ",per)


Percent of data dropped is  11.109999999999998


In [ ]:
#We will use ordinal encoder, because there is an order in the categorical variables.
#This is used from inbuilt sklearn packages.
#This will encode the categorical values into numerical features, but will leave the missing values as it is.
#We use KNN imputer, which takes the effect of nearest k - datapoints and impute their weighted average.
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import KNNImputer
#instantiate both packages to use
encoder = OrdinalEncoder()
# create a list of categorical columns to iterate over
cat_cols = ['platform_id', 'language']

def encode(data):
    '''function to encode non-null data and replace it in the original data'''
    #retains only non-null values
    nonulls = np.array(data.dropna())
    #reshapes the data for encoding
    impute_reshape = nonulls.reshape(-1,1)
    #encode date
    impute_ordinal = encoder.fit_transform(impute_reshape)
    #Assign back encoded values to non-null values
    data.loc[data.notnull()] = np.squeeze(impute_ordinal)
    return data

#create a for loop to iterate through each column in the data
for columns in cat_cols:
    encode(concatenated_df[columns])

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
concatenated_df #This is the encoded data, with missing values.

,song_id,platform_id,released_year,language,no.of_comments,label_id
0,3753,0.0,2005.0,6,882,5.6
1,9019,1.0,1999.0,5,178,3677.8
2,7173,2.0,1935.0,6,759,51773.3
3,43,3.0,1847.0,6,31212,20781.8
4,5452,4.0,2000.0,5,2441,275870.5
...,...,...,...,...,...,...
14,8055,4038.0,NaN,NaN,NaN,308963.9
15,8477,5003.0,NaN,NaN,NaN,280243.83
16,8665,5145.0,NaN,NaN,NaN,513049.99
17,9534,6383.0,NaN,NaN,NaN,463858.84


In [ ]:
#Now, we will impute missing values, using knn imputer, as said above.
imputer = KNNImputer(n_neighbors=1000, weights= 'distance')
concatenated_df_imputed = imputer.fit_transform(concatenated_df) #This has fully imputed data of song features.
#This is in form of an array. It must be converted to dataframe.

In [ ]:
song_features_not_sorted = pd.DataFrame(concatenated_df_imputed)
song_features_not_sorted.rename(columns = {0: 'song_id', 1:'platform_id', 2:'released_year', 3:'language',4:'no.of_comments',5:'label_id' }, inplace = True)
song_features_not_sorted #This has song features imputed, as a dataframe.

,song_id,platform_id,released_year,language,no.of_comments,label_id
0,3753.0,0.0,2005.000000,6.000000,882.000000,5.60
1,9019.0,1.0,1999.000000,5.000000,178.000000,3677.80
2,7173.0,2.0,1935.000000,6.000000,759.000000,51773.30
3,43.0,3.0,1847.000000,6.000000,31212.000000,20781.80
4,5452.0,4.0,2000.000000,5.000000,2441.000000,275870.50
...,...,...,...,...,...,...
9995,8055.0,4038.0,1975.227709,5.971204,806.955559,308963.90
9996,8477.0,5003.0,1970.994758,5.961104,848.666886,280243.83
9997,8665.0,5145.0,1978.116370,6.030922,1108.225227,513049.99
9998,9534.0,6383.0,1957.222015,5.824262,1013.893743,463858.84


In [ ]:
song_features = sortdf(song_features_not_sorted, 0)
song_features #This is a dataframe wit hsong features, fully imputed, sorted as per song_id.

,song_id,platform_id,released_year,language,no.of_comments,label_id
0,1.0,2481.0,2008.0,6.000000,155254.0,9267070.80
1,2.0,3505.0,1997.0,6.000000,75867.0,2187458.10
2,3.0,5633.0,2005.0,5.000000,95009.0,8099955.90
3,4.0,7499.0,1960.0,6.000000,72586.0,2800147.10
4,5.0,1688.0,1925.0,6.000000,51992.0,6650830.80
...,...,...,...,...,...,...
9995,9996.0,6864.0,2010.0,6.000000,1180.0,1847961.40
9996,9997.0,4333.0,1990.0,6.000000,395.0,1056549.20
9997,9998.0,1883.0,1977.0,6.000000,374.0,861782.86
9998,9999.0,3905.0,2011.0,6.000000,1988.0,7323064.50


#4. **Standardising Data**

In [ ]:
#X_standard is standardised dataframe
#We will not standardise song_id, as it is merely an indicator.
numerical_columns= ['released_year','no.of_comments', 'label_id','platform_id','language']
categorical_columns =['song_id']

X_standard =pd.DataFrame([])

################## Standarizing values only for numerical columns################
from sklearn.preprocessing import StandardScaler

X_standard[numerical_columns] =pd.DataFrame(StandardScaler().fit_transform(song_features[numerical_columns]))

################## combining categorical columns ################
X_standard[categorical_columns]=song_features[categorical_columns]
X_standard 

,released_year,no.of_comments,label_id,platform_id,language,song_id
0,0.171803,24.875979,2.957586,-0.872434,0.213034,1.0
1,0.098426,11.911939,-0.261011,-0.517710,0.213034,2.0
2,0.151791,15.037862,2.426982,0.219451,-0.611240,3.0
3,-0.148385,11.376145,0.017536,0.865852,0.213034,4.0
4,-0.381855,8.013108,1.768168,-1.147137,0.213034,5.0
...,...,...,...,...,...,...
9995,0.185144,-0.284583,-0.415356,0.645882,0.213034,9996.0
9996,0.051732,-0.412775,-0.775154,-0.230882,0.213034,9997.0
9997,-0.034985,-0.416205,-0.863701,-1.079587,0.213034,9998.0
9998,0.191814,-0.152635,2.073784,-0.379146,0.213034,9999.0


# 5. **Training code**

In [ ]:
#we will consider customer based filtering first. In this, we train separate models for individual customers based on their rated songs.
#d_c is a dictionary with keys as customer_id's and values as the list of tuples of (song_id, customer_score) from train data.
#It has 14053 keys.
d_c = {}

customer_set = set(train_data['customer_id'])

# There are 14053 customer_ids in the above set

for i in customer_set :
  d_c[i] = []

arr_t = np.array(train_data)
sz_t = np.shape(arr_t)[0]

for i in range(0, sz_t):
  d_c[arr_t[i][0]].append((arr_t[i][1], arr_t[i][2]))

In [ ]:
#Reordering the columns of X_standard for ease.
X_standard = X_standard[['song_id', 'released_year', 'no.of_comments', 'label_id', 'platform_id', 'language']]
X_standard
#Creating a copy of X_standard for further analysis.
song_features_svm = X_standard.copy()
song_features_svm #copy of X_standard

,song_id,released_year,no.of_comments,label_id,platform_id,language
0,1.0,0.171803,24.875979,2.957586,-0.872434,0.213034
1,2.0,0.098426,11.911939,-0.261011,-0.517710,0.213034
2,3.0,0.151791,15.037862,2.426982,0.219451,-0.611240
3,4.0,-0.148385,11.376145,0.017536,0.865852,0.213034
4,5.0,-0.381855,8.013108,1.768168,-1.147137,0.213034
...,...,...,...,...,...,...
9995,9996.0,0.185144,-0.284583,-0.415356,0.645882,0.213034
9996,9997.0,0.051732,-0.412775,-0.775154,-0.230882,0.213034
9997,9998.0,-0.034985,-0.416205,-0.863701,-1.079587,0.213034
9998,9999.0,0.191814,-0.152635,2.073784,-0.379146,0.213034


In [ ]:
#We write below a function svm_f1, that takes song_id as int and gives its features from song_features_svm 
#This gives features of a song 
def svm_f1(song_id) :
  i = int(float(song_id)-1)
  res = [float(song_features_svm['released_year'][i]), float(song_features_svm['no.of_comments'][i]), float(song_features_svm['label_id'][i]), song_features_svm['platform_id'][i], song_features_svm['language'][i]]
  return res

In [ ]:
#Now, we will construct a personalised train dataset for each customer. 
#For every one of 14053 customers, we have 6 features.
#They are the song_id, platform_id, label_id, released_year, language, no.of.comments.
#"score" is the target.
#This function gives a dataframe, given a customer_id. 


def dataset(customer_id):
  D_c = {}

  D_c['released_year'] = []
  D_c['no.of_comments'] = []
  D_c['platform_id'] = []
  D_c['label_id'] = []
  D_c['language'] = []
  D_c['score'] = []

  v = d_c[customer_id]
  for i in v:

      song_feat = svm_f1(int(i[0]))
      D_c['released_year'].append(float(song_feat[0]))
      D_c['no.of_comments'].append(float(song_feat[1]))
      D_c['label_id'].append(float(song_feat[2]))
      D_c['platform_id'].append(str(song_feat[3]))
      D_c['language'].append(str(song_feat[4]))
      D_c['score'].append(float(i[1]))

  X_c = pd.DataFrame(D_c)
  return X_c

In [ ]:
#For once and all, all the training datasets of 14053 customers are pulled out from train_data and 
#stored into this dictionary(matrix_customer), with keys as customer_id's and values as the datasets (pd.Dataframes)
#This is done to reduce time complexity for later use.
matrix_customer = {}
for j in d_c:
  matrix_customer[j] = dataset(j)

In [ ]:
#similar to training data, we now pull the data from test set also, corresponding to a customer.
#This is done using a similar approach, in a dict d_test.
#We use this to make personalised predictions.
d_test = {}

customer_set_test = set(test['customer_id'])
print(len(customer_set_test))

for i in customer_set_test :
  d_test[i] = []

arr_t = np.array(test)
sz_t = np.shape(arr_t)[0]

for i in range(0, sz_t):
  d_test[arr_t[i][1]].append((arr_t[i][0], arr_t[i][2], arr_t[i][3]))

14053


In [ ]:
#Now, we will construct a personalised test dataset for each customer. 
#They are the song_id, platform_id, label_id, released_year, language, no.of.comments
#Score is filled with np.nans. THis is done for later ease.
#These datasets are used to make personalised predictions.


def dataset_for_test(customer_id):
  D_c = {}
  # D_c['song_id'] = []
  D_c['released_year'] = []
  D_c['no.of_comments'] = []
  D_c['platform_id'] = []
  D_c['label_id'] = []
  D_c['language'] = []
  D_c['score'] = []
  D_c['test_row_id'] = []
  # flag = 0
  # if pr[customer_id]:
  #   flag = 1
  #   D_c['saving_probability'] = []
  v = d_test[customer_id]
  for i in v:
      # D_c['song_id'].append(float(i[0]))
      song_feat = svm_f1(int(i[1]))
      D_c['released_year'].append(float(song_feat[0]))
      D_c['no.of_comments'].append(float(song_feat[1]))
      D_c['label_id'].append(float(song_feat[2]))
      D_c['platform_id'].append(str(song_feat[3]))
      D_c['language'].append(str(song_feat[4]))
      D_c['score'].append(i[2])
      D_c['test_row_id'].append(i[0]) 

      # if flag == 1:
      #   cluster = song_features_svm['cluster_code'][int(i[0])-1]
      #   D_c['saving_probability'].append(customer_saving_prob[customer_id][cluster])
  X_c = pd.DataFrame(D_c)
  return X_c

In [ ]:
#For once and all, all the training datasets of 14053 customers are pulled out from train_data and 
#stored into this dictionary, with keys as customer_id's and values as the datasets (pd.Dataframes)
matrix_customer_test = {}
for j in d_test:
  matrix_customer_test[j] = dataset_for_test(j)

In [ ]:
#Initilaising an empty pd.Dataframe "prediction" with "columns" list below as its columns. 
#We will fill predictions from each customer's model on the customer's test set, obtained from above.
#To retain the order of rows in test.csv, we add the row number as a feature.
columns = ['test_row_id','score']
prediction = pd.DataFrame(columns = columns)
prediction

,test_row_id,score


#6.**Making Predictions**

In [ ]:
# #matrix_customer is for train
# #matrix_customer_test is for test
#We will pull out the train and test matrices for each customer, append them properly.
#we will get an appended data frame with 7 columns i.e. six song_features and score.
#THe test set has score values np.nan
#We use KNN imputing for filling the scores. THis is also called KNN regression.
#We append the results from each test data to the predictions data frame, along with the point's test_row_id.
#Later, we will sort it using sortdf function written above, to sort according to test_row_id.
count = 0
for p in customer_set:
  test_matrix = matrix_customer_test[p].drop(['test_row_id'], axis =1)
  train_matrix = matrix_customer[p].copy()
  l = len(train_matrix)
  l2 = len(test_matrix)
  gh = train_matrix.append(test_matrix)
  imputer = KNNImputer(n_neighbors=l, weights= 'distance')
  customer_matrix_imp = imputer.fit_transform(gh)
  z = pd.DataFrame(customer_matrix_imp[l:len(customer_matrix_imp)])
  z.rename(columns = {0: 'released_year', 1:'no.of_comments', 2:'platform_id', 3:'label_id',4:'language',5:'score' }, inplace = True)
  columns = ['test_row_id','score']
  lo = pd.DataFrame(columns = columns)
  lo['score'] = z['score']
  lo['test_row_id'] = matrix_customer_test[p]['test_row_id']
  prediction = prediction.append(lo)
  count = count + 1






In [ ]:
#Sorting the prediction with the test_row_id
result = sortdf(prediction, 0)

In [ ]:
result #The sorted prediction

,test_row_id,score
0,0.0,3.722657
1,1.0,3.788888
2,2.0,3.804747
3,3.0,3.659282
4,4.0,4.496959
...,...,...
670676,670676.0,3.431370
670677,670677.0,3.955928
670678,670678.0,4.030610
670679,670679.0,4.202718


In [ ]:
#The result exported to csv file.
# result.to_csv('submission_7.csv')

#7.Collaborative Filtering

1.   Previously, I made predictions using the customer dataset alone.
2.   This may induce bias and also, if the custoemr data is not sufficient, accuracy is compromised.
3. Now, we use collective option of all customers on a song.
4. We take the train data, take the average of ratings of all customers to a given song, and assign this value to it for any customer in the test set, for this song.
5. This takes into account of all customers and since there are all 10k songs in the train data, we have one score for a given song_id, independent of its other features.



In [ ]:
#Bring all the features into train_set
#Trim the customer_id part. So we will get columns with song_features and scores.
train_data_trimmed =train_data.drop(['customer_id'], axis = 1)
train_data_trimmed
sorted_train_data = sortdf(train_data_trimmed, 0)
sorted_train_data

,song_id,score
0,1,4
1,1,3
2,1,4
3,1,4
4,1,4
...,...,...
710104,10000,4
710105,10000,4
710106,10000,5
710107,10000,3


In [ ]:
#Initialise a list "ratings", where index i corresponds to the rating of song_id -i , as defined above.
#Here i = 1 to 10000.
#ratings[0] is a redundant element.
#These are average ratings from train set. Other song_features and datasets are not needed here.
ratings = [0]*10001
count = 1
ratings[sorted_train_data['song_id'][0]] = sorted_train_data['score'][0]
for x in range(1,len(sorted_train_data)):
  if sorted_train_data['song_id'][x] == sorted_train_data['song_id'][x-1]:
    ratings[sorted_train_data['song_id'][x]] += sorted_train_data['score'][x]
    count = count+1
  else:
    ratings[sorted_train_data['song_id'][x-1]] /= count
    count = 1
    ratings[sorted_train_data['song_id'][x]] = sorted_train_data['score'][x]


In [ ]:
#ratings[i] has ratings of song_id - i
#test_copy has the required prediction of song scores.
#It has 2 columns, test_row_id and scores. scores are from ratings list defined above.
results = []
for k in test['song_id']:
  results.append(ratings[k])
test_copy = test.copy()
test_copy = test_copy.drop(columns = ["customer_id","song_id"])
test_copy['score'] = np.array(results)

In [ ]:
#Exporting test_copy as csv file. 
#test_copy.to_csv("only_collaborative.csv")

#8. Hybrid Approach 

1.   Now, we will combine the results from both above approaches. This will make the predictions better.
2.   We take mean of both the above predictions for every data point in test set and report them as final predictions.



In [ ]:
#We convert scores from both above methods into arrays, take their mean.
#We add the socres into "final_predictions" dataframe. 
#It has 2 columns, test_row_id, and score as needed.
#This is the final prediction.
a = np.array(test_copy['score'])
b = np.array(result['score'])
c = (a+b)/2
final_predictions = test_copy.copy()
final_predictions['score'] = c 

In [ ]:
#Exporting the final results to csv
final_predictions.to_csv('final_predictions.csv')

# THE CODE BLOCK BELOW THIS IS AN APPROACH TRIED, BUT IT IS DISCARDED DUE TO SLIGHT INCREASE IN ERROR. BUT WE WANT TO SHOW OUR EFFORTS.

#9. Including saved lists by Clustering of songs.

We **tried** to accommodate saved_for_later songs into the model by clustering the songs with their features into optimal 5 clusters and finding the probability that a customer saves a cluster songs using this dataset. This is done by adding a feature called saving probability. But this increased the error slightly, so we discarded it.

In [ ]:
X_standard

,song_id,released_year,no.of_comments,label_id,platform_id,language
0,1.0,0.171803,24.875979,2.957586,-0.872434,0.213034
1,2.0,0.098426,11.911939,-0.261011,-0.517710,0.213034
2,3.0,0.151791,15.037862,2.426982,0.219451,-0.611240
3,4.0,-0.148385,11.376145,0.017536,0.865852,0.213034
4,5.0,-0.381855,8.013108,1.768168,-1.147137,0.213034
...,...,...,...,...,...,...
9995,9996.0,0.185144,-0.284583,-0.415356,0.645882,0.213034
9996,9997.0,0.051732,-0.412775,-0.775154,-0.230882,0.213034
9997,9998.0,-0.034985,-0.416205,-0.863701,-1.079587,0.213034
9998,9999.0,0.191814,-0.152635,2.073784,-0.379146,0.213034


In [ ]:
from sklearn.cluster import KMeans
X_no_song_id = X_standard.drop(["song_id"], axis = 1)
kmeans = KMeans(n_clusters=5, random_state=0).fit(X_no_song_id)


In [ ]:
centroids = kmeans.cluster_centers_
cluster_labels = kmeans.labels_
opt_error = kmeans.inertia_

In [ ]:
X_standard['cluster_labels'] = cluster_labels

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
d = {} #THis is dict for saved
sortedset = sortdf(saveforlater, 0)
s4 = sorted(set(saveforlater['customer_id']))
for i in s4:
  d[i] = []
  
arr_svl = np.array(sortedset)
sz_svl = np.shape(arr_svl)[0]

for i in range(0,sz_svl) :
  d[arr_svl[i][0]].append(int(arr_svl[i][1]))

In [ ]:
len(d)
d['F10040']

[342, 7275, 1136, 4893, 4958, 188, 165, 1167, 1796]

In [ ]:
customer_saving_prob = {} #A dict of keys as customer names of 12799 customers, and values as their saving probabilities.
for j in d:
  customer_saving_prob[j] = [0]*5 
  for k in d[j]:
    customer_saving_prob[j][X_standard['cluster_labels'][int(k)-1]] += 1
  for m in range(len(customer_saving_prob[j])):
    customer_saving_prob[j][m] /= len(d[j])


In [ ]:
Set1 = set(train_data['customer_id'])
pr = {}
for i in Set1:
  pr[i] = False
Set2 = d.keys()
for j in Set2:
  pr[j] = True

In [ ]:
def dataset_with_saving(customer_id):
  X_c = matrix_customer[customer_id]
  flag = 0
  if pr[customer_id]:
    flag = 1
    # X_c['saving_probability'] = np.array([])
    z = []
  v = d_c[customer_id]
  for i in v:
      if flag == 1:
        cluster = X_standard['cluster_labels'][int(i[0])-1]
        z.append(customer_saving_prob[customer_id][cluster])
  if flag == 1:
    X_c['saving_probability'] = np.array(z)
    X_c = X_c[['released_year', 'no.of_comments', 'platform_id', 'label_id','language', 'saving_probability', 'score']]
  return X_c

In [ ]:
def dataset_with_saving_test(customer_id):
  X_c = matrix_customer_test[customer_id]
  flag = 0
  if pr[customer_id]:
    flag = 1
    # X_c['saving_probability'] = np.array([])
    z = []
  v = d_test[customer_id]
  for i in v:
      if flag == 1:
        cluster = X_standard['cluster_labels'][int(i[1])-1]
        z.append(customer_saving_prob[customer_id][cluster])
  if flag == 1:
    X_c['saving_probability'] = np.array(z)
    X_c = X_c[['released_year', 'no.of_comments', 'platform_id', 'label_id','language', 'saving_probability', 'score','test_row_id']]
  return X_c

In [ ]:
matrix_customer_withsaving = {}
for j in d_c:
  matrix_customer_withsaving[j] = dataset_with_saving(j)


matrix_customer_test_withsaving = {}
for j in d_test:
  matrix_customer_test_withsaving[j] = dataset_with_saving_test(j)

In [ ]:
columns = ['test_row_id','score']
prediction_withsaving = pd.DataFrame(columns = columns)
prediction_withsaving

,test_row_id,score


In [ ]:
#Modifying the prediction algo to include the saving probabilities
#matrix_customer is for train
#matrix_customer_test is for test
# len(customer_set)
count = 0
# p = "F10040"
# if p == "F10040" :

for p in customer_set:
  count = count + 1
  # print("Appending " + p + " customer " + str(count))
  test_matrix_withsaving = matrix_customer_test_withsaving[p].drop(['test_row_id'], axis =1)
  train_matrix_withsaving = matrix_customer_withsaving[p].copy()
  l = len(train_matrix_withsaving)
  l2 = len(test_matrix_withsaving)
  gh = train_matrix_withsaving.append(test_matrix_withsaving)
  imputer = KNNImputer(n_neighbors=l, weights= 'distance')
  customer_matrix_imp = imputer.fit_transform(gh)
  # if len(customer_matrix_imp)!= l + l2:
  #   print("length not compatible")
  #   print(len(customer_matrix_imp))
  #   break
  z = pd.DataFrame(customer_matrix_imp[l:len(customer_matrix_imp)])
  if(pr[p]):
    z.rename(columns = {0: 'released_year', 1:'no.of_comments', 2:'platform_id', 3:'label_id',4:'language',5:'saving_probability',6:'score' }, inplace = True)
  else:
    z.rename(columns = {0: 'released_year', 1:'no.of_comments', 2:'platform_id', 3:'label_id',4:'language',5:'score' }, inplace = True)
  columns = ['test_row_id','score']
  lo = pd.DataFrame(columns = columns)
  lo['score'] = z['score']
  lo['test_row_id'] = matrix_customer_test_withsaving[p]['test_row_id']
  prediction_withsaving = prediction_withsaving.append(lo)



In [ ]:
# prediction_withsaving.to_csv('prediction_withsaving_without_others.csv')